In [1]:
from IPython.display import display
 
import pandas as pd

%matplotlib inline

In [2]:
df = pd.read_parquet('../data/craiglist_cars.parquet')
 
display(df.info())
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434542 entries, 0 to 434541
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          434542 non-null  int64 
 1   manufacturer  418698 non-null  object
 2   condition     274367 non-null  object
 3   cylinders     311539 non-null  object
 4   fuel          430894 non-null  object
 5   title_status  431661 non-null  object
 6   transmission  430244 non-null  object
 7   drive         376834 non-null  object
 8   size          181927 non-null  object
 9   type          384280 non-null  object
 10  paint_color   348787 non-null  object
 11  price         434542 non-null  int64 
dtypes: int64(2), object(10)
memory usage: 39.8+ MB


None

,year,manufacturer,condition,cylinders,fuel,title_status,transmission,drive,size,type,paint_color,price
0,2009,chevrolet,good,8 cylinders,gas,clean,automatic,rwd,full-size,SUV,white,9000
1,2002,gmc,good,8 cylinders,gas,clean,automatic,4wd,None,pickup,white,6000
2,2007,pontiac,excellent,4 cylinders,gas,clean,automatic,fwd,compact,convertible,red,7000


In [3]:
def percentage_nas(df):
    empty_percentage = df.isna().sum()/df.shape[0]
    empty_percentage = empty_percentage[empty_percentage > 0]
 
    return empty_percentage
 
empty = percentage_nas(df)
columns_more_empty = empty[empty >= 0.5].index
df.drop(columns=columns_more_empty, inplace=True)
 
percentage_nas(df)

manufacturer    0.036461
condition       0.368606
cylinders       0.283064
fuel            0.008395
title_status    0.006630
transmission    0.009891
drive           0.132802
type            0.115667
paint_color     0.197346
dtype: float64

In [4]:
 
mode = df['cylinders'].mode()[0]
 
df['cylinders'].fillna(mode, inplace=True)
df['cylinders'] = df['cylinders'].replace({'other' : mode})
df['cylinders'] = df['cylinders'].replace( regex={r'\s+\w+$' : '',}).astype(int)

In [5]:
df_categorical = df.select_dtypes('object')
df.drop(columns=df_categorical.columns, inplace=True)

In [6]:
for column in df_categorical:
    df_categorical[column].fillna(df_categorical[column].mode()[0], inplace=True)
 
display(df_categorical.nunique())
percentage_nas(df_categorical)

manufacturer    42
condition        6
fuel             5
title_status     6
transmission     3
drive            3
type            13
paint_color     12
dtype: int64

Series([], dtype: float64)

In [7]:
df = pd.concat([df, pd.get_dummies(df_categorical).copy()], axis=1)
 
display(df.dtypes.value_counts())
display(df.info())
df.head(3)

uint8    90
int64     2
int32     1
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434542 entries, 0 to 434541
Data columns (total 93 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   year                          434542 non-null  int64
 1   cylinders                     434542 non-null  int32
 2   price                         434542 non-null  int64
 3   manufacturer_acura            434542 non-null  uint8
 4   manufacturer_alfa-romeo       434542 non-null  uint8
 5   manufacturer_aston-martin     434542 non-null  uint8
 6   manufacturer_audi             434542 non-null  uint8
 7   manufacturer_bmw              434542 non-null  uint8
 8   manufacturer_buick            434542 non-null  uint8
 9   manufacturer_cadillac         434542 non-null  uint8
 10  manufacturer_chevrolet        434542 non-null  uint8
 11  manufacturer_chrysler         434542 non-null  uint8
 12  manufacturer_datsun           434542 non-null  uint8
 13  manufacturer_d

None

,year,cylinders,price,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,...,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
0,2009,8,9000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2002,8,6000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2007,4,7000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [8]:
df.to_parquet('../data/cars_v1.parquet')